In [13]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from io import BytesIO
import gradio as gr

In [21]:
load_dotenv(override=True)
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [22]:
client_oai = OpenAI(api_key=openai_api_key)
client_ds = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

In [23]:
system_prompt = """
你是一位面试官，而我准备参加小学数学教师招聘面试，需试讲人教版小学数学某节选片段，试讲分导入、新授、练习巩固、小结、作业五部分（必须有这五个部分且必须按照以下顺序）。接下来我会上传我的试讲内容，请用中文点评我的试讲表现，给出我的缺点，并告诉我如何改正。  
1.导入：生动有趣，具教育性，体现学科融合与时政（如：“同学们，喜欢《西游记》吗？最爱谁？孙悟空？对，他会七十二变！看大屏幕，金箍棒时长时短，像新闻里的伸缩科技，怎么量它？今天我们学习毫米、分米！”）。  
2.新授：目标明确，重点突出，符合课标与教材，内容准确，结构完整，挖掘智能与思想因素，难易适度，步骤清晰，重难点处理得当，方法灵活，启发性强，主导与主体结合，思维活跃，反馈良好。  
3.练习巩固：帮助学生对新授的内容进行巩固练习，承上启下，有趣，面向全体，全班订正。  
4.小结：知识性与思想性兼备。  
5.注意：融入多样教学法（如小组讨论、小游戏、情境如“买东西找零”），积极鼓励，互动频繁，时长10-15分钟。  
最后提供板书设计（突出重点）。"""

In [24]:
def add_message(history, message):
    for x in message["files"]:
        # 检查是否是音频文件（假设支持常见格式如 .wav, .mp3）
        if x.lower().endswith((".wav", ".mp3", ".m4a")):
            with open(x, "rb") as audio_file:
                transcription = client_oai.audio.transcriptions.create(
                    model="whisper-1", 
                    file=audio_file
                )
            history.append({"role": "user", "content": transcription.text})

    if message["text"] is not None:
        history.append({"role": "user", "content": message["text"]})
    return history, gr.MultimodalTextbox(value=None, interactive=False)

In [25]:
def oai_4omini_bot(history: list):
    messages = [{"role": "system", "content": system_prompt}] + history
    stream = client_oai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True)
    history.append({"role": "assistant", "content": ""})
    full_response = ''
    for chunk in stream:
        full_response += chunk.choices[0].delta.content or ''   
        history[-1]["content"] = full_response 
        yield history

In [26]:
def oai_4o_bot(history: list):
    messages = [{"role": "system", "content": system_prompt}] + history
    stream = client_oai.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        stream=True)
    history.append({"role": "assistant", "content": ""})
    full_response = ''
    for chunk in stream:
        full_response += chunk.choices[0].delta.content or ''   
        history[-1]["content"] = full_response 
        yield history

In [27]:
def ds_v3_bot(history: list):
    messages = [{"role": "system", "content": system_prompt}] + history
    stream = client_ds.chat.completions.create(
        model='deepseek-chat',
        messages=messages,
        stream=True)
    history.append({"role": "assistant", "content": ""})
    full_response = ''
    for chunk in stream:
        full_response += chunk.choices[0].delta.content or ''   
        history[-1]["content"] = full_response 
        yield history

In [28]:
def ds_r1_bot(history: list):
    messages = [{"role": "system", "content": system_prompt}] + history
    stream = client_ds.chat.completions.create(
        model='deepseek-reasoner',
        messages=messages,
        stream=True)
    history.append({"role": "assistant", "content": ""})
    full_response = ''
    for chunk in stream:
        full_response += chunk.choices[0].delta.content or ''   
        history[-1]["content"] = full_response 
        yield history

In [29]:
def bot(history, model_choice):
    # 根据 model_choice 调用不同的 bot 函数
    if model_choice == "GPT 4o-mini":
        yield from oai_4omini_bot(history)  # 使用 yield from 传递生成器结果
    elif model_choice == "GPT 4o":
        yield from oai_4o_bot(history)
    elif model_choice == "DeepSeek V3":
        yield from ds_v3_bot(history)
    elif model_choice == "DeepSeek R1":
        yield from ds_r1_bot(history)

In [30]:
with gr.Blocks() as demo:
    gr.Markdown("""# Welcom Teacher Xie!👩🏻‍🏫
                **让我们一起来练习吧~🤺**
    """)
    chatbot = gr.Chatbot(elem_id="这是陈老师做的", bubble_full_width=False, type="messages")

    model_choice = gr.Dropdown(
                label="选择模型",
                choices=["GPT 4o-mini", "GPT 4o", "DeepSeek V3", "DeepSeek R1"],
                value="GPT 4o-mini"  # 默认选择 
            )

    chat_input = gr.MultimodalTextbox(
        interactive=True,
        file_count="multiple",
        placeholder="Enter message or upload file...",
        show_label=False,
        sources=["microphone", "upload"],
    )

    chat_msg = chat_input.submit(add_message, [chatbot, chat_input], [chatbot, chat_input])
    bot_msg = chat_msg.then(bot, [chatbot, model_choice], chatbot, api_name="bot_response")
    bot_msg.then(lambda: gr.MultimodalTextbox(interactive=True), None, [chat_input])

demo.launch(share=True)

C:\Users\zekin\AppData\Local\Temp\ipykernel_78368\3652135178.py:5: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(elem_id="这是陈老师做的", bubble_full_width=False, type="messages")


* Running on local URL:  http://127.0.0.1:7916


* Running on public URL: https://7e7cc61ff4d0f610eb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
